# Crypto Altcoin Analysis
This workbook will walk through extracting data from Coinbase on all altcoins available with a USD pairing, then clean data and do some performance and correlation analysis.



In [1]:
# Imports
import panel as pn
pn.extension('plotly')
import plotly.express as px
import numpy as np
import pandas as pd
#from Historic_Crypto import HistoricalData
import hvplot.pandas
#import matplotlib.pyplot as plt
import os
from pathlib import Path
from dotenv import load_dotenv
import csv
import cbpro
import requests
import time

ModuleNotFoundError: No module named 'panel'

In [ ]:
# Setup Public Client for Coinbase
public_client = cbpro.PublicClient()

In [ ]:
# Pull a list of all currencies (Assets and pairings)
currency_response_list = public_client.get_currencies()

In [ ]:
# Pull only the assets that are marked as crypto

# Create empty list of cryptos
crypto_list = []

# If the asset is labeled as a crypto add to crypto list
for asset in currency_response_list:
    if asset['details']['type'] == 'crypto':
        crypto_list.append(asset['id'])
crypto_list.sort()
print(crypto_list)
print(len(crypto_list))

In [ ]:
# Get a list of all Coinbase Products
all_products_list = public_client.get_products()
print(len(all_products_list))
#all_products_list

In [ ]:
# Find product ids that have a USD quote_currency and create a list

# Create Empty List for USD paired products
usd_product_id_list = []
for product in all_products_list:
    if product['quote_currency'] == 'USD':
        usd_product_id_list.append(product['id'])
usd_product_id_list.sort()
cleaned_product_list = []
[cleaned_product_list.append(x) for x in usd_product_id_list if x not in cleaned_product_list]
print(cleaned_product_list)
print(type(cleaned_product_list))
print(len(cleaned_product_list))

In [ ]:
# Make calls to the Products endpoint for each product_Id and pull candlestick data for the last 300 weeks
# GET https://api.pro.coinbase.com/products/<product-id>/candles
# Reponses contain the following metrics in a list of lists.
# [0] time bucket start time
# [1] low lowest price during the bucket interval
# [2] high highest price during the bucket interval
# [3] open opening price (first trade) in the bucket interval
# [4] close closing price (last trade) in the bucket interval
# [5] volume volume of trading activity during the bucket interval
# columns1 = ['Date'] + usd_product_id_list
main_df = pd.DataFrame()

# Apparently only some of these product ID's have data going back to 2019 - these are those
good_list = ['ALGO-USD', 'BCH-USD', 'BTC-USD', 'DASH-USD', 'EOS-USD', 'ETC-USD', 'ETH-USD', 'LINK-USD', 'LTC-USD', 'REP-USD', 'UMA-USD', 'XLM-USD', 'XTZ-USD', 'ZRX-USD']

for item in good_list:
    historical = pd.DataFrame(public_client.get_product_historic_rates(item, start='2019-01-01T00:00:00-06:00', end='2019-10-19T00:00:00-05:00', granularity=86400))
    historical.columns=["Date","Open","High","Low",item,"Volume"]
    historical['Date'] = pd.to_datetime(historical['Date'], unit='s')
    historical.set_index('Date', inplace=True)
    historical.sort_values(by='Date', ascending=True, inplace=True)
    historical = historical.drop(['Open', 'High', 'Low', 'Volume'], axis=1)
    main_df = pd.concat([main_df, historical], axis=1)
    time.sleep(10)
print(main_df)

    

In [ ]:
# Cleaning assets with less than 2 years
